In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")


In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

races_schema= StructType(
    fields = [
        StructField('raceId', IntegerType(), False),
        StructField('year', IntegerType(), True),
        StructField('round', IntegerType(), True),
        StructField('circuitId', IntegerType(), False),
        StructField('name', StringType(), True),
        StructField('date', StringType(), True),
        StructField('time', StringType(), True),
        StructField('url', StringType(), True),
    ]
)

In [0]:
races_df = spark.read\
    .option('header', True)\
    .schema(races_schema)\
    .csv(f'{raw_folder_path}/{v_file_date}/races.csv')

In [0]:
from pyspark.sql.functions import to_timestamp, concat, lit, col

races_timestamp_df = races_df\
.withColumn('race_timestamp', to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))\
.withColumn("data_source", lit(v_data_source)) \
.withColumn("file_date", lit(v_file_date))

In [0]:
races_selected_df = races_timestamp_df.select(
    col('raceId').alias('race_id'),
    col('year').alias('race_year'),
    col('round'),
    col('circuitId').alias('circuit_id'), 
    col('name'), 
    col('race_timestamp'),
    col('data_source'),
    col('file_date')
    )

In [0]:
 races_final_df = add_ingestion_date(races_selected_df)

In [0]:
overwrite_partition(races_final_df,'race_year', 'races', 'f1_processed')


In [0]:
dbutils.notebook.exit("Success")